<link rel="stylesheet" href="/site-assets/css/gemma.css">
<link rel="stylesheet" href="https://fonts.googleapis.com/css2?family=Google+Symbols:opsz,wght,FILL,GRAD@20..48,100..700,0..1,-50..200" />

# Xây một con chatbot với Gemma 2

Large Language Models (LLMs) như Gemma thì làm tốt việc generating informative responses, nên mình sẽ xài nó để build trợ lý ảo với chatbot.

Thường thì mấy LLM sẽ hoạt động trong một stateless manner, tức là nó không có inherent memory để lưu trữ đoạn hội thoại cũ. Mỗi prompt hay mỗi question sẽ được processed độc lập, không liên quan tới những tương tác trước đó. Nhưng mà một điều quan trọng của natural conversation là phải giữ được context trước đó. Để vượt qua hạn chế này và cho phép LLM maintain được conversation context (or pertinent parts), LLM phải dược cung cấp một cách rõ ràng các thông tin có liên quan chẳng hạn như lịch sử cuộc trò chuyện cho mỗi một prompt mới được present đến LLM.

Đây là tutorial để build một con chatbot, xài instruction-tuned model variant của Gemma.

## Setup

### Gemma setup

* Tìm Gemma model card trên kaggle.com, request access và điền consent form để kaggle duyệt quyền xài model
* Chọn Colab Runtime phù hợp, ở chỗ góc cao bên phải của Google Colab, chọn T4 GPU
* Vào mục Public key của Kaggle để download file json có chứa username và API key để add vào trong mục secrets của Colab

### Set environment variables cho `KAGGLE_USERNAME` và `KAGGLE_KEY`.

In [ ]:
# Note: `userdata.get` là một Colab API. Nếu không dùng Colab thì set env
import os
from google.colab import userdata

os.environ["KAGGLE_USERNAME"] = "hngtrnvminh"
os.environ["KAGGLE_KEY"] = "9d6234a4e515964fd29db30c0d0367d1"

### Tải Keras với KerasNLP

In [ ]:
# Tải Keras 3.  https://keras.io/getting_started/
# tensorflow-cpu để đảm bảo tính tương thích với mấy thư viện tensorflow-specific, nhưng mà mặc định thì sẽ xài gpu nếu có, mà google colab thì có T4
# keras-nlp nó có sẵn kha khá model và tool được prebuilt như là GemmaCausalLM, thì lúc triển khai nó dễ hơn
# tensorflow hub với text để thêm tiện ích cho text processing với pretrained model
!pip install -q tensorflow-cpu
!pip install -q -U keras-nlp tensorflow-hub
!pip install -q -U "keras>=3"
!pip install -q -U tensorflow-text

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.2/230.2 MB 5.1 MB/s eta 0:00:00


### Lựa backend

Keras là một high-level, multi-framework deep learning API thiết kế hướng đến sự đơn giản và dễ xài và sẽ cho mình chọn một trong 3 backend: TensorFlow, JAX, or PyTorch. Cả 3 đều sẽ work được trong cái tutorial này hết

In [ ]:
import os

# Chọn Jax làm backend
# Jax cung cấp khả năng tính toán hiệu quả trên accelerator như là TPU với GPU kiểu công nghiệp, ở đây mình có GPU xịn của Google nên mình sẽ xài Jax
# So với pytorch thì jax cũng cho tốc độ tốt hơn trong nhiều trường hợp, đặc biệt là trên TPU GPU chuyên dụng
# Với các dự án dùng consumer-grade gpu như là mấy cái rtx trên laptop thì mình nên compile bằng torch tại tương thích sẽ tốt hơn
os.environ["KERAS_BACKEND"] = "jax"

# Pre-allocate 100% TPU memory để giảm thiểu phân mảnh(memory fragmentation) với tăng perf
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.0"

### Import Keras, KerasNLP và set seed



In [ ]:
import keras
import keras_nlp

# reproducibility này là kiểu chỉ số để đảm bảo con chatbot nó không trả lời quá khác nhau giữa những lần hỏi khác nhau mà cùng nội dung á
# cái số 42 nó hong có gì đặc biệt đâu kiểu là để model biết là sự ngẫu nhiên giữa những câu trả lời cho mỗi câu hỏi giống nhau sẽ không quá nhiều miễn là cái seed này được set ở 42
keras.utils.set_random_seed(42)

### Khởi tạo model

KerasNLP cung cấp rất nhiều implementation cho các [model architectures](https://keras.io/api/keras_nlp/models/) nổi bật. Trong notebook này thì mình sẽ khởi tạo model bằng `GemmaCausalLM`, một model Gemma end-to-end cho causal language modeling. Một causal language model thì dự đoán token tiếp theo dựa trên những token trước đó.

Khởi tạo model bằng method `from_preset`:

In [ ]:
#Xài kerasnlp để load model, này là giải pháp cực kì optimized và được đóng gói sẵn hết
#khỏi phải lo về mấy cái setup phức tạp như là AutoModelForCausalLM hay là tự build thủ công tokenizer với model, giảm bớt rất nhiều bottleneck
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_2b_en")

 `GemmaCausalLM.from_preset()` function khởi tạo model từ một preset architecture và weights. String `"gemma2_instruct_2b_en"` chỉ cụ thể preset the Gemma 2 2B model với 2 tỉ parameters. Model Gemma với [7B, 9B, and 27B parameters](https://ai.google.com/gemma/docs/get_started#models-list) cũng available. Có thể tìm thấy code string cho mấy model đó ở  **Model Variation** listings trên [Kaggle](https://www.kaggle.com/models/google/gemma).


Xài `summary` method để lấy thông tin về model:

In [ ]:
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

Từ summary thấy được model có 2.6 tỉ trainable parameters.


### Define formatting helper functions

In [ ]:
# Này là mình format chat output, kiểu nhìn nó như nào á, ở đây mình format bằng Markdown trong thư viện IPython của Jupyter
from IPython.display import Markdown
import textwrap

def display_chat(prompt, text):
  formatted_prompt = "<font size='+1' color='brown'>🙋‍♂️<blockquote>" + prompt + "</blockquote></font>"
  text = text.replace('•', '  *')
  text = textwrap.indent(text, '> ', predicate=lambda _: True)
  formatted_text = "<font size='+1' color='teal'>🤖\n\n" + text + "\n</font>"
  return Markdown(formatted_prompt+formatted_text)

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

## Build chatbot

Gemma instruction-tuned model `gemma2_instruct_2b_en` được fine-tuned để hiểu được turn tokens như sau:

```
<start_of_turn>user\n  ... <end_of_turn>\n
<start_of_turn>model\n ... <end_of_turn>\n
```

Tutorial cũng dùng những tokens đó để build chatbot. Xem thêm ở [Formatting and system instructions](https://ai.google.dev/gemma/docs/formatting)


### Tạo chat helper để quản lý conversation state

In [ ]:
# Đây là main logic để quản lý cuộc hội thoại của chatbot
# System and History Management: Tracks messages bằng markers (<start_of_turn> và <end_of_turn>) cho cả user và model
# Prompt Construction: Builds một conversation context hoàn chỉnh (get_full_prompt) bằng cách kết hợp cả history và system instructions
# Gửi message: Thêm user input vào history -> build prompt -> generate ra response bằng generate method của model -> update history với phản hồi từ model
class ChatState():

  __START_TURN_USER__ = "<start_of_turn>user\n"
  __START_TURN_MODEL__ = "<start_of_turn>model\n"
  __END_TURN__ = "<end_of_turn>\n"

  def __init__(self, model, system=""):

    self.model = model
    self.system = system
    self.history = []

  def add_to_history_as_user(self, message):

      self.history.append(self.__START_TURN_USER__ + message + self.__END_TURN__)

  def add_to_history_as_model(self, message):

      self.history.append(self.__START_TURN_MODEL__ + message)

  def get_history(self):

      return "".join([*self.history])

  def get_full_prompt(self):

    prompt = self.get_history() + self.__START_TURN_MODEL__
    if len(self.system)>0:
      prompt = self.system + "\n" + prompt
    return prompt

  def send_message(self, message):

    self.add_to_history_as_user(message)
    prompt = self.get_full_prompt()
    response = self.model.generate(prompt, max_length=2048)
    result = response.replace(prompt, "")
    self.add_to_history_as_model(result)
    return result


### Chat với model

Bắt đầu chat với model.

In [ ]:
# Khởi tạo chatbot và gửi query đến chatbot
chat = ChatState(gemma_lm)
message = "Explain Riemann hypothesis to me like I'm a high schoooler"
display_chat(message, chat.send_message(message))

<font size='+1' color='brown'>🙋‍♂️<blockquote>Explain Riemann hypothesis to me like I'm a high schoooler</blockquote></font><font size='+1' color='teal'>🤖

> Imagine you have a giant, endless sheet of paper.  You want to fill it with numbers, but there's a catch: you can only use whole numbers.  
> 
> Now, imagine you're trying to find all the "prime numbers" on this sheet. Prime numbers are like the special, unique building blocks of numbers. They can only be divided by 1 and themselves.  
> 
> The Riemann Hypothesis is about understanding the distribution of these prime numbers on this paper. It's a big, complex mathematical puzzle that scientists have been trying to solve for over a century.
> 
> Here's the gist:
> 
> * **Prime Numbers:**  Prime numbers are like the special, unique building blocks of numbers. They can only be divided by 1 and themselves.
> * **The Riemann Zeta Function:** This is a special mathematical function that helps us understand the distribution of prime numbers. It's like a map that shows us where prime numbers are likely to appear.
> * **The Riemann Hypothesis:** This hypothesis states that the distribution of prime numbers follows a very specific pattern, and this pattern can be described by the Riemann Zeta Function.
> 
> **Why is it so important?**
> 
> * **Cryptography:** Prime numbers are the foundation of modern encryption, which keeps our online information safe. Understanding the Riemann Hypothesis could help us develop even stronger encryption methods.
> * **Number Theory:** The Riemann Hypothesis is a fundamental problem in number theory, the study of numbers and their properties. Solving it would revolutionize our understanding of numbers.
> * **Mathematics in general:**  It's a big deal because it connects different areas of mathematics, like algebra, geometry, and analysis.
> 
> **The Challenge:**
> 
> The Riemann Hypothesis is incredibly difficult to prove. It's like trying to solve a puzzle with millions of pieces, and even the most brilliant mathematicians have struggled to crack it.
> 
> **In a nutshell:**
> 
> The Riemann Hypothesis is a big, unsolved puzzle about prime numbers and their distribution. It's a fascinating problem that could have major implications for cryptography, mathematics, and our understanding of the universe. 
> 
> 
> Let me know if you'd like me to explain any of these concepts in more detail! 
> <end_of_turn>
</font>

In [ ]:
message = "Gimme code to compute all prime numbers up to 100"
display_chat(message, chat.send_message(message))

<font size='+1' color='brown'>🙋‍♂️<blockquote>Gimme code to compute all prime numbers up to 100</blockquote></font><font size='+1' color='teal'>🤖

> ```python
> def is_prime(num):
>   """Checks if a number is prime."""
>   if num <= 1:
>     return False
>   if num <= 3:
>     return True
>   if num % 2 == 0 or num % 3 == 0:
>     return False
>   i = 5
>   while i * i <= num:
>     if num % i == 0 or num % (i + 2) == 0:
>       return False
>     i += 6
>   return True
> 
> primes = []
> for num in range(2, 101):
>   if is_prime(num):
>     primes.append(num)
> 
> print("Prime numbers up to 100:", primes)
> ```
> 
> **Explanation:**
> 
> 1. **`is_prime(num)` function:**
>    - Takes an integer `num` as input.
>    - Returns `True` if `num` is prime, `False` otherwise.
>    - Optimizations:
>      - Handles numbers less than or equal to 3 directly.
>      - Checks divisibility by 2 and 3 first.
>      - Uses the 6k ± 1 optimization: All prime numbers greater than 3 can be expressed in the form 6k ± 1 (where k is any integer).
> 
> 2. **Main loop:**
>    - Iterates through numbers from 2 to 100.
>    - For each number, calls the `is_prime()` function.
>    - If the function returns `True`, the number is added to the `primes` list.
> 
> 3. **Output:**
>    - Prints the list of prime numbers found.
> 
> **How it works:**
> 
> The code efficiently checks for primality by:
> 
> - **Early optimization:**  It handles special cases for numbers less than or equal to 3.
> - **6k ± 1 optimization:**  It checks divisibility only by numbers of the form 6k ± 1, significantly reducing the number of checks.
> 
> 
> Let me know if you'd like to explore other prime number algorithms or have any more questions! 
> <end_of_turn>
</font>

In [ ]:
message = "Great! Now add those explanations as comments in the code."
display_chat(message, chat.send_message(message))

<font size='+1' color='brown'>🙋‍♂️<blockquote>Great! Now add those explanations as comments in the code.</blockquote></font><font size='+1' color='teal'>🤖

> ```python
> def is_prime(num):
>   """Checks if a number is prime.
> 
>   Args:
>     num: The number to check.
> 
>   Returns:
>     True if num is prime, False otherwise.
>   """
>   if num <= 1:  # Numbers less than or equal to 1 are not prime
>     return False
>   if num <= 3:  # 2 and 3 are prime
>     return True
>   if num % 2 == 0 or num % 3 == 0:  # Check divisibility by 2 and 3
>     return False
>   i = 5  # Start checking divisibility from 5
>   while i * i <= num:  # Check divisibility up to the square root of num
>     if num % i == 0 or num % (i + 2) == 0:  # If divisible by i or i+2, not prime
>       return False
>     i += 6  # Increment by 6 to check the next potential divisors
>   return True  # If no divisors found, it's prime
> 
> primes = []  # Initialize an empty list to store prime numbers
> for num in range(2, 101):  # Iterate through numbers from 2 to 100
>   if is_prime(num):  # Check if the number is prime
>     primes.append(num)  # If prime, add it to the list
> 
> print("Prime numbers up to 100:", primes)  # Print the list of prime numbers
> ``` 
> 
> 
> Let me know if you have any other questions! 
> <end_of_turn>
</font>

Test thử code mà model đưa luôn:

In [ ]:
def is_prime(num):
  """Checks if a number is prime.

  Args:
    num: The number to check.

  Returns:
    True if num is prime, False otherwise.
  """
  if num <= 1:  # Numbers less than or equal to 1 are not prime
    return False
  if num <= 3:  # 2 and 3 are prime
    return True
  if num % 2 == 0 or num % 3 == 0:  # Check divisibility by 2 and 3
    return False
  i = 5  # Start checking divisibility from 5
  while i * i <= num:  # Check divisibility up to the square root of num
    if num % i == 0 or num % (i + 2) == 0:  # If divisible by i or i+2, not prime
      return False
    i += 6  # Increment by 6 to check the next potential divisors
  return True  # If no divisors found, it's prime

primes = []  # Initialize an empty list to store prime numbers
for num in range(2, 101):  # Iterate through numbers from 2 to 100
  if is_prime(num):  # Check if the number is prime
    primes.append(num)  # If prime, add it to the list

print("Prime numbers up to 100:", primes)  # Print the list of prime numbers

Prime numbers up to 100: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


In [ ]:
import time
from IPython.display import display  # Ensures Markdown rendering

def timed_chat(message):
    start_time = time.time()
    response = chat.send_message(message)
    total_time = time.time() - start_time
    tokens_generated = len(response.split())  # Approximate token count
    tokens_per_sec = tokens_generated / total_time if total_time > 0 else 0

    # ✅ Ensure chat response is displayed properly
    display(display_chat(message, response))

    # ✅ Print generation stats
    print(f"\n⏱️ Total Generation Time: {total_time:.3f} sec | ⚡ Tokens/sec: {tokens_per_sec:.2f}")


In [ ]:
timed_chat("Explain Riemann hypothesis to me like I'm a high schooler")

<font size='+1' color='brown'>🙋‍♂️<blockquote>Explain Riemann hypothesis to me like I'm a high schooler</blockquote></font><font size='+1' color='teal'>🤖

> Okay, imagine a giant, endless sheet of paper.  You want to fill it with numbers, but there's a catch: you can only use whole numbers.  
> 
> Now, imagine you're trying to find all the "prime numbers" on this sheet. Prime numbers are like the special, unique building blocks of numbers. They can only be divided by 1 and themselves. Think of 2, 3, 5, 7, 11... those are prime numbers.
> 
> The Riemann Hypothesis is about understanding how these prime numbers are spread out on this paper. It's a big, complex mathematical puzzle that scientists have been trying to solve for over a century.
> 
> Here's the gist:
> 
> * **Prime Numbers:** Prime numbers are like the special, unique building blocks of numbers. They can only be divided by 1 and themselves.
> * **The Riemann Zeta Function:** This is a special mathematical function that helps us understand the distribution of prime numbers. It's like a map that shows us where prime numbers are likely to appear.
> * **The Riemann Hypothesis:** This hypothesis states that the distribution of prime numbers follows a very specific pattern, and this pattern can be described by the Riemann Zeta Function.
> 
> **Why is it so important?**
> 
> * **Cryptography:** Prime numbers are the foundation of modern encryption, which keeps our online information safe. Understanding the Riemann Hypothesis could help us develop even stronger encryption methods.
> * **Number Theory:** The Riemann Hypothesis is a fundamental problem in number theory, the study of numbers and their properties. Solving it would revolutionize our understanding of numbers.
> * **Mathematics in general:**  It's a big deal because it connects different areas of mathematics, like algebra, geometry, and analysis.
> 
> **The Challenge:**
> 
> The Riemann Hypothesis is incredibly difficult to prove. It's like trying to solve a puzzle with millions of pieces, and even the most brilliant mathematicians have struggled to crack it.
> 
> **In a nutshell:**
> 
> The Riemann Hypothesis is a big, unsolved puzzle about prime numbers and their distribution. It's a fascinating problem that could have major implications for cryptography, mathematics, and our understanding of the universe. 
> 
> 
> Let me know if you'd like me to explain any of these concepts in more detail! 
> <end_of_turn>
</font>


⏱️ Total Generation Time: 29.055 sec | ⚡ Tokens/sec: 12.25


Xài `get_history` method để xem context được retained bởi `Chat` class như nào

In [ ]:
print(chat.get_history())

<start_of_turn>user
Tell me, in a few words,  how to compute all prime numbers up to 1000?<end_of_turn>
<start_of_turn>model
**Sieve of Eratosthenes.** 
<end_of_turn><start_of_turn>user
Now in Python! No numpy, please!<end_of_turn>
<start_of_turn>model
```python
def sieve_of_eratosthenes(n):
  """Returns a list of prime numbers up to n."""
  primes = [True] * (n + 1)
  primes[0] = primes[1] = False
  for i in range(2, int(n**0.5) + 1):
    if primes[i]:
      for j in range(i * i, n + 1, i):
        primes[j] = False
  return [i for i, is_prime in enumerate(primes) if is_prime]

primes = sieve_of_eratosthenes(1000)
print(primes)
```

**Explanation:**

1. **Initialization:**
   - `primes = [True] * (n + 1)`: Creates a list `primes` of boolean values, initially assuming all numbers are prime.
   - `primes[0] = primes[1] = False`: Sets 0 and 1 as non-prime.

2. **Iteration:**
   - `for i in range(2, int(n**0.5) + 1):`: Iterates from 2 to the square root of `n`. We only need to check up to

## Tổng kết và đọc thêm

Vậy là xong phần build chatbot với Gemma 2B Instruction tuned model xài Keras on JAX.

Đọc thêm:

* [Get started with Keras Gemma](https://ai.google.dev/gemma/docs/get_started).
* [Finetune the Gemma model on GPU](https://ai.google.dev/gemma/docs/lora_tuning).
* [Gemma integration with Vertex AI](https://ai.google.dev/gemma/docs/integrations/vertex)
* [Gemma on Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/open-models/use-gemma).


## Ưu điểm của setup sử dụng Google Colab
* *Ease of setup*: Code và dependencies minimal hơn, trong khi Local thì cần nhiều setup hơn cho tokenizer, model và server. keras_nlp.models.GemmaCausalLM.from_preset nó load trực tiếp nguyên một cái pre-trained model đã được tối ưu, không cần phải lo thêm về handling, memory optimization hay set up tokenizer
* *Harware Uitilization*: Tương thích với T4 GPU(qua Jax), Local thì cần cấu hình thủ công cho CPU và GPU
* *Model loading*: Pre-trained thông qua preset của Keras-NLP, Local thì cần loading thủ công thông qua transformers, cần nhiều VRAM hơn
* *Framework*: Jax(nhanh hơn và tối ưu trên TPU GPU chuyên dụng)
* *UI*: Colab(Markdown trong Colab, module IPython của Jupyter gọn nhẹ)
* *Tốc độ Inference*: Nhờ Jax nên rất tối ưu là một, Keras-NLP lại còn lo hết từ batching tới memory tới token generation nữa nên siêu tối ưu, còn giải pháp Local thì cần tự làm batching, cũng kha khá phức tạp và tăng batch size lên thì cũng cần nhiều vram, với mình thử tự set batch_size thì lỗi nhảy đỏ lè nên hong làm:>
* *Backend*: Colab không có server overhead luôn mà là direct execution, mọi thứ nó chạy trong hết cùng 1 process.